In [ ]:
```json
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# ChromaMood: Data Exploration"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Setup and Imports"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import cv2\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "from sklearn.cluster import KMeans\n",
    "from collections import Counter\n",
    "import os\n",
    "import time\n",
    "from IPython.display import display, clear_output"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Data Loading and Inspection"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### 2.1 Load Static Image"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def load_image(image_path):\n",
    "    if not os.path.exists(image_path):\n",
    "        print(f\"Error: Image file not found at {image_path}\")\n",
    "        print(\"Please provide a valid image path or place 'sample_image.jpg' in the same directory.\")\n",
    "        # Create a dummy black image as a fallback\n",
    "        return np.zeros((100, 100, 3), dtype=np.uint8), False\n",
    "    \n",
    "    image = cv2.imread(image_path)\n",
    "    if image is None:\n",
    "        print(f\"Error: Could not read image file at {image_path}\")\n",
    "        return np.zeros((100, 100, 3), dtype=np.uint8), False\n",
    "        \n",
    "    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Convert to RGB for consistency\n",
    "    return image, True\n",
    "\n",
    "# IMPORTANT: Replace 'sample_image.jpg' with your image path if needed\n",
    "static_image_path = 'sample_image.jpg' \n",
    "static_image, loaded_successfully = load_image(static_image_path)\n",
    "\n",
    "if loaded_successfully:\n",
    "    print(f\"Loaded image: {static_image_path}\")\n",
    "    print(f\"Image shape: {static_image.shape}\")\n",
    "    print(f\"Image data type: {static_image.dtype}\")\n",
    "    \n",
    "    plt.figure(figsize=(6, 6))\n",
    "    plt.imshow(static_image)\n",
    "    plt.title('Static Image Preview')\n",
    "    plt.axis('off')\n",
    "    plt.show()\n",
    "else:\n",
    "    print(\"Skipping static image analysis due to loading error.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### 2.2 Webcam Feed Access (Helper Function)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def get_webcam_frame(cap):\n",
    "    ret, frame = cap.read()\n",
    "    if not ret:\n",
    "        print(\"Error: Failed to capture frame from webcam.\")\n",
    "        return None\n",
    "    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)\n",
    "    return frame\n",
    "\n",
    "# Example: Capture one frame for inspection (optional)\n",
    "# cap = cv2.VideoCapture(0)\n",
    "# if not cap.isOpened():\n",
    "#     print(\"Error: Could not open webcam.\")\n",
    "# else:\n",
    "#     print(\"Webcam opened successfully.\")\n",
    "#     frame = get_webcam_frame(cap)\n",
    "#     if frame is not None:\n",
    "#         print(f\"Captured frame shape: {frame.shape}\")\n",
    "#         plt.figure(figsize=(6, 4))\n",
    "#         plt.imshow(frame)\n",
    "#         plt.title('Webcam Frame Preview')\n",
    "#         plt.axis('off')\n",
    "#         plt.show()\n",
    "#     cap.release()\n",
    "#     print(\"Webcam released.\")\n",
    "# print(\"Webcam check complete.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Exploratory Data Analysis (Color Extraction & Visualization)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### 3.1 Dominant Color Extraction (K-Means)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def get_dominant_colors(image, k=5, image_processing_size=None):\n",
    "    \"\"\"Extracts k dominant colors from an image using K-Means clustering.\"\"\"\n",
    "    if image is None:\n",
    "        return [], []\n",
    "\n",
    "    # Resize image for faster processing if specified\n",
    "    if image_processing_size is not None:\n",
    "        image = cv2.resize(image, image_processing_size, interpolation=cv2.INTER_AREA)\n",
    "\n",
    "    # Reshape the image to be a list of pixels\n",
    "    pixels = image.reshape((-1, 3))\n",
    "    pixels = np.float32(pixels)\n",
    "\n",
    "    # Perform K-Means clustering\n",
    "    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)\n",
    "    kmeans.fit(pixels)\n",
    "\n",
    "    # Get the cluster centers (dominant colors) and labels\n",
    "    colors = kmeans.cluster_centers_\n",
    "    labels = kmeans.labels_\n",
    "\n",
    "    # Count labels to find the frequency of each cluster\n",
    "    counts = Counter(labels)\n",
    "\n",
    "    # Sort colors by frequency\n",
    "    sorted_counts = sorted(counts.items(), key=lambda item: item[1], reverse=True)\n",
    "    dominant_colors = []\n",
    "    percentages = []\n",
    "    total_pixels = pixels.shape[0]\n",
    "\n",
    "    for i, count in sorted_counts:\n",
    "        dominant_colors.append(colors[i].astype(int).tolist())\n",
    "        percentages.append((count / total_pixels) * 100)\n",
    "\n",
    "    return dominant_colors, percentages"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### 3.2 Visualize Dominant Colors (Palette Plot)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def plot_color_palette(colors, percentages, title='Dominant Color Palette'):\n",
    "    \"\"\"Plots the extracted colors as a horizontal bar.\"\"\"\n",
    "    if not colors:\n",
    "        print(\"No colors to plot.\")\n",
    "        return\n",
    "        \n",
    "    bar_height = 1\n",
    "    plt.figure(figsize=(8, 2))\n",
    "    \n",
    "    # Normalize percentages to sum to 1 for width calculation\n",
    "    total_percentage = sum(percentages)\n",
    "    normalized_percentages = [p / total_percentage for p in percentages]\n",
    "    \n",
    "    current_width = 0\n",
    "    for i, color in enumerate(colors):\n",
    "        width = normalized_percentages[i]\n",
    "        plt.barh(0, width, left=current_width, height=bar_height, color=np.array(color)/255.0)\n",
    "        current_width += width\n",
    "        \n",
    "    plt.title(title)\n",
    "    plt.yticks([])\n",
    "    plt.xticks([])\n",
    "    plt.ylim(-bar_height/2, bar_height/2)\n",
    "    plt.xlim(0, 1)\n",
    "    plt.tight_layout()\n",
    "    plt.show()\n",
    "\n",
    "# Analyze static image\n",
    "if loaded_successfully:\n",
    "    print(f\"Analyzing static image: {static_image_path}\")\n",
    "    # Use a smaller size for faster K-Means on potentially large images\n",
    "    static_dom_colors, static_percentages = get_dominant_colors(static_image, k=5, image_processing_size=(100, 100))\n",
    "    print(\"Dominant Colors (RGB):\", static_dom_colors)\n",
    "    print(\"Percentages:\", [f\"{p:.1f}%\" for p in static_percentages])\n",
    "    plot_color_palette(static_dom_colors, static_percentages, title=f'Static Image Palette ({static_image_path})')\n",
    "else:\n",
    "    print(\"Skipping static image color analysis.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### 3.3 Color Distribution (HSV Histogram)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def plot_hsv_histograms(image, title='HSV Color Distribution'):\n",
    "    if image is None:\n",
    "        print(\"Cannot plot histogram for None image.\")\n",
    "        return\n",
    "        \n",
    "    hsv_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)\n",
    "    h, s, v = cv2.split(hsv_image)\n",
    "    \n",
    "    fig, axs = plt.subplots(1, 3, figsize=(15, 4))\n",
    "    fig.suptitle(title)\n",
    "    \n",
    "    # Hue Histogram (0-179 for OpenCV)\n",
    "    axs[0].hist(h.ravel(), bins=180, range=[0, 180], color='red')\n",
    "    axs[0].set_title('Hue')\n",
    "    axs[0].set_xlim([0, 180])\n",
    "    \n",
    "    # Saturation Histogram (0-255)\n",
    "    axs[1].hist(s.ravel(), bins=256, range=[0, 256], color='green')\n",
    "    axs[1].set_title('Saturation')\n",
    "    axs[1].set_xlim([0, 256])\n",
    "    \n",
    "    # Value Histogram (0-255)\n",
    "    axs[2].hist(v.ravel(), bins=256, range=[0, 256], color='blue')\n",
    "    axs[2].set_title('Value')\n",
    "    axs[2].set_xlim([0, 256])\n",
    "    \n",
    "    plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Adjust layout to prevent title overlap\n",
    "    plt.show()\n",
    "\n",
    "# Plot histogram for static image\n",
    "if loaded_successfully:\n",
    "    plot_hsv_histograms(static_image, title=f'Static Image HSV Distribution ({static_image_path})')\n",
    "else:\n",
    "    print(\"Skipping static image histogram.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Statistical Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def analyze_colors_statistically(colors, percentages):\n",
    "    if not colors:\n",
    "        return {'mean_rgb': [0,0,0], 'std_rgb': [0,0,0], 'mean_hsv': [0,0,0], 'weighted_mean_rgb': [0,0,0]}\n",
    "        \n",
    "    np_colors = np.array(colors)\n",
    "    np_percentages = np.array(percentages) / 100.0 # Normalize percentages\n",
    "    \n",
    "    # Basic RGB stats\n",
    "    mean_rgb = np.mean(np_colors, axis=0)\n",
    "    std_rgb = np.std(np_colors, axis=0)\n",
    "    \n",
    "    # Weighted RGB mean (by percentage)\n",
    "    weighted_mean_rgb = np.average(np_colors, axis=0, weights=np_percentages)\n",
    "    \n",
    "    # Convert dominant colors to HSV for analysis\n",
    "    # Need to reshape for cvtColor: (1, N, 3)\n",
    "    num_colors = np_colors.shape[0]\n",
    "    rgb_for_hsv = np_colors.reshape(1, num_colors, 3).astype(np.uint8)\n",
    "    hsv_colors = cv2.cvtColor(rgb_for_hsv, cv2.COLOR_RGB2HSV).reshape(num_colors, 3)\n",
    "    \n",
    "    mean_hsv = np.mean(hsv_colors, axis=0)\n",
    "    \n",
    "    return {\n",
    "        'mean_rgb': mean_rgb.tolist(),\n",
    "        'std_rgb': std_rgb.tolist(),\n",
    "        'mean_hsv': mean_hsv.tolist(), # [Mean H, Mean S, Mean V]\n",
    "        'weighted_mean_rgb': weighted_mean_rgb.tolist()\n",
    "    }\n",
    "\n",
    "# Analyze stats for static image dominant colors\n",
    "if loaded_successfully and static_dom_colors:\n",
    "    color_stats = analyze_colors_statistically(static_dom_colors, static_percentages)\n",
    "    print(\"\\n--- Statistical Analysis of Dominant Colors (Static Image) ---\")\n",
    "    print(f\"Mean RGB: {color_stats['mean_rgb']}\")\n",
    "    print(f\"Std Dev RGB: {color_stats['std_rgb']}\")\n",
    "    print(f\"Weighted Mean RGB: {color_stats['weighted_mean_rgb']}\")\n",
    "    print(f\"Mean HSV: {color_stats['mean_hsv']}\")\n",
    "else:\n",
    "    print(\"\\nSkipping statistical analysis for static image.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Feature Engineering Experiments"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### 5.1 Color Feature: Warmth/Coolness"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def get_color_warmth(rgb_color):\n",
    "    \"\"\"Calculates a simple warmth score based on RGB values.\"\"\"\n",
    "    r, g, b = rgb_color\n",
    "    # Simple heuristic: more Red/Yellow = warmer, more Blue = cooler\n",
    "    warmth = (r - b) # Difference between red and blue\n",
    "    # Normalize roughly to a -255 to 255 range\n",
    "    return warmth\n",
    "\n",
    "def get_palette_warmth(colors, percentages):\n",
    "    \"\"\"Calculates the overall warmth of a palette, weighted by percentage.\"\"\"\n",
    "    if not colors:\n",
    "        return 0\n",
    "        \n",
    "    total_warmth = 0\n",
    "    total_percentage = sum(percentages)\n",
    "    if total_percentage == 0: return 0\n",
    "    \n",
    "    for color, percentage in zip(colors, percentages):\n",
    "        warmth = get_color_warmth(color)\n",
    "        total_warmth += warmth * (percentage / total_percentage)\n",
    "        \n",
    "    return total_warmth\n",
    "\n",
    "# Calculate warmth for static image palette\n",
    "if loaded_successfully and static_dom_colors:\n",
    "    palette_warmth = get_palette_warmth(static_dom_colors, static_percentages)\n",
    "    print(f\"\\nPalette Warmth Score (Static Image): {palette_warmth:.2f}\")\n",
    "    if palette_warmth > 50:\n",
    "        print(\"Overall impression: Warm\")\n",
    "    elif palette_warmth < -30:\n",
    "        print(\"Overall impression: Cool\")\n",
    "    else:\n",
    "        print(\"Overall impression: Neutral\")\n",
    "else:\n",
    "     print(\"\\nSkipping warmth calculation for static image.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### 5.2 Mood Mapping"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def suggest_mood(colors, percentages):\n",
    "    \"\"\"Suggests a mood based on dominant colors and their properties.\"\"\"\n",
    "    if not colors:\n",
    "        return \"Undefined\"\n",
    "\n",
    "    stats = analyze_colors_statistically(colors, percentages)\n",
    "    warmth = get_palette_warmth(colors, percentages)\n",
    "    mean_h, mean_s, mean_v = stats['mean_hsv']\n",
    "    \n",
    "    # Simple rule-based mood suggestion\n",
    "    if warmth > 60 and mean_s > 100 and mean_v > 100:\n",
    "        return \"Energetic / Passionate\"\n",
    "    elif warmth > 30 and mean_s > 80 and mean_v > 120:\n",
    "        return \"Happy / Joyful\"\n",
    "    elif warmth < -40 and mean_s > 50 and mean_v > 80:\n",
    "        return \"Calm / Serene\"\n",
    "    elif warmth < -20 and mean_s < 70 and mean_v < 100:\n",
    "        return \"Melancholic / Peaceful\"\n",
    "    elif mean_s < 50 and mean_v > 150: # Low saturation, high value\n",
    "        return \"Soft / Gentle\"\n",
    "    elif mean_s < 40 and mean_v < 80: # Low saturation, low value\n",
    "        return \"Mysterious / Subdued\"\n",
    "    else:\n",
    "        return \"Neutral / Balanced\"\n",
    "\n",
    "# Suggest mood for static image\n",
    "if loaded_successfully and static_dom_colors:\n",
    "    suggested_mood = suggest_mood(static_dom_colors, static_percentages)\n",
    "    print(f\"Suggested Mood (Static Image): {suggested_mood}\")\n",
    "else:\n",
    "    print(\"\\nSkipping mood suggestion for static image.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Initial Model Testing & Application"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### 6.1 Abstract Art Generation"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def generate_abstract_art(colors, percentages, width=400, height=200, style='stripes'):\n",
    "    \"\"\"Generates a simple abstract image based on the color palette.\"\"\"\n",
    "    if not colors:\n",
    "        return np.zeros((height, width, 3), dtype=np.uint8)\n",
    "\n",
    "    art_image = np.zeros((height, width, 3), dtype=np.uint8)\n",
    "    total_percentage = sum(percentages)\n",
    "    if total_percentage == 0: return art_image # Avoid division by zero\n",
    "    normalized_percentages = [p / total_percentage for p in percentages]\n",
    "\n",
    "    if style == 'stripes':\n",
    "        current_x = 0\n",
    "        for i, color in enumerate(colors):\n",
    "            stripe_width = int(normalized_percentages[i] * width)\n",
    "            if i == len(colors) - 1: # Ensure last stripe fills remaining width\n",
    "                 stripe_width = width - current_x\n",
    "            \n",
    "            end_x = current_x + stripe_width\n",
    "            # Ensure color is in BGR for OpenCV drawing if needed, but here we use numpy slicing with RGB\n",
    "            art_image[:, current_x:end_x] = color \n",
    "            current_x = end_x\n",
    "            \n",
    "    elif style == 'blocks':\n",
    "        # Simple block layout (e.g., 2xN grid)\n",
    "        num_colors = len(colors)\n",
    "        cols = (num_colors + 1) // 2\n",
    "        rows = 2 if num_colors > 1 else 1\n",
    "        block_h = height // rows\n",
    "        block_w = width // cols\n",
    "        color_idx = 0\n",
    "        for r in range(rows):\n",
    "            for c in range(cols):\n",
    "                if color_idx < num_colors:\n",
    "                    y1, y2 = r * block_h, (r + 1) * block_h\n",
    "                    x1, x2 = c * block_w, (c + 1) * block_w\n",
    "                    # Handle last block potentially extending\n",
    "                    if r == rows - 1: y2 = height\n",
    "                    if c == cols - 1: x2 = width\n",
    "                    art_image[y1:y2, x1:x2] = colors[color_idx]\n",
    "                    color_idx += 1\n",
    "                    \n",
    "    return art_image\n",
    "\n",
    "# Generate art for static image\n",
    "if loaded_successfully and static_dom_colors:\n",
    "    art_stripes = generate_abstract_art(static_dom_colors, static_percentages, style='stripes')\n",
    "    art_blocks = generate_abstract_art(static_dom_colors, static_percentages, style='blocks')\n",
    "\n",
    "    fig, axs = plt.subplots(1, 2, figsize=(12, 4))\n",
    "    axs[0].imshow(art_stripes)\n",
    "    axs[0].set_title('Abstract Art (Stripes)')\n",
    "    axs[0].axis('off')\n",
    "    axs[1].imshow(art_blocks)\n",
    "    axs[1].set_title('Abstract Art (Blocks)')\n",
    "    axs[1].axis('off')\n",
    "    plt.tight_layout()\n",
    "    plt.show()\n",
    "else:\n",
    "    print(\"\\nSkipping abstract art generation for static image.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### 6.2 Real-time Webcam Analysis (Conceptual Example)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# This cell demonstrates the flow but real-time display in Jupyter is limited.\n",
    "# It will print updates and show the final state.\n",
    "# For true real-time, a dedicated OpenCV window loop (`cv2.imshow`) outside Jupyter is better.\n",
    "\n",
    "run_webcam_analysis = False # Set to True to run this section\n",
    "\n",
    "if run_webcam_analysis:\n",
    "    cap = cv2.VideoCapture(0)\n",
    "    if not cap.isOpened():\n",
    "        print(\"Error: Could not open webcam.\")\n",
    "    else:\n",
    "        print(\"Starting webcam analysis... Press 'q' in the output window or interrupt kernel to stop.\")\n",
    "        \n",
    "        fig, axs = plt.subplots(1, 3, figsize=(18, 5)) # Frame, Palette, Art\n",
    "        \n",
    "        try:\n",
    "            frame_count =